Task1_0603. train.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

In [ ]:
# Name 컬럼 Unique 확인
for name in df.Name:
  if 'Mr' in name:
    pass
  elif 'Miss' in name:
    pass
  elif 'Mrs' in name:
    pass
  else:
    print(name)

# Age 컬럼 null값 끄집어내기
df.Age.unique()
df.Age.isnull().info()
df[df.Age.isnull()].Sex

# 남/여 별 나이 평균
round(df.groupby('Sex').Age.mean().female)
round(df.groupby('Sex').Age.mean().male)

# fare 값의 분포를 확인하고 이에 따라 pclass 컬럼 재정의
df['Fare'].isnull().sum()

# 히스토그램 생성
plt.figure(figsize=(10, 6))
plt.hist(df.Fare, bins=30, color='blue', alpha=0.9)
plt.title('Fare Distribution')
plt.xlabel('Fare')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

df.Fare.describe()

# 0, 5, 10, 15, 20, 30, 50, 75, 100, 200, 600 : 10개 분류

In [161]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [162]:
# 파일불러오기
file_path = 'drive/MyDrive/KDT_2404/m3_분석 라이브러리/dataset/train.csv'
df = pd.read_csv(file_path)

# 결측값이 많은 컬럼, 필요없는 컬럼 삭제
df.drop(columns=['PassengerId', 'Pclass', 'Ticket', 'Cabin', 'Name'], inplace=True)
df.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,1,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,0,male,35.0,0,0,8.0500,S


In [163]:
# Embarked 결측치 처리
df['Embarked'].fillna('S', inplace=True)

# sibsp와 parch 합쳐서 가족 규모 변수 생성
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1 # 자기 자신 포함
df.drop(columns=['SibSp', 'Parch'], inplace=True)

# Age 결측치 : Sex 별 평균으로 결측치 처리
age_mean_by_sex = round(df.groupby('Sex').Age.transform('mean'))
df['Age'].fillna(age_mean_by_sex, inplace=True)

# Age를 6개 범주로 확대
bins = [0, 5, 12, 18, 35, 60, 100]
labels = [1, 2, 3, 4, 5, 6]
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels)
df.drop(columns=['Age'], inplace=True)

# Sex 컬럼 정수 변환
df.Sex.replace({'female':0, 'male':1}, inplace=True)

# Embarked를 더미 변수로 변환
df = pd.get_dummies(df, columns=['Embarked'])
#df.Embarked.replace({'S': 0, 'C': 1, 'Q': 2}, inplace=True)

# Fare를 10개 범주로 확대
fare_bins = [0, 5, 10, 15, 20, 30, 50, 75, 100, 200, 600]
fare_labels = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
df['FareGroup'] = pd.cut(df['Fare'], bins=fare_bins, labels=fare_labels, include_lowest=True)
df.drop(columns=['Fare'], inplace=True)

# FareGroup별 남성의 비율
df['SbF'] = df.groupby('FareGroup').Sex.transform('mean')

In [159]:
# Embarked by Faregroup
#df['AgeGroup'] = df.AgeGroup.astype(int)
#df['FareGroup'] = df.FareGroup.astype(int)

#df['EbF'] = df.groupby('FareGroup').Embarked.transform('mean')
#df['EbF'] = df.Embarked / df.FareGroup
#df['EbF']
#df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Survived    891 non-null    int64   
 1   Sex         891 non-null    int64   
 2   FamilySize  891 non-null    int64   
 3   AgeGroup    891 non-null    int64   
 4   Embarked_C  891 non-null    bool    
 5   Embarked_Q  891 non-null    bool    
 6   Embarked_S  891 non-null    bool    
 7   FareGroup   891 non-null    category
 8   SbF         891 non-null    float64 
dtypes: bool(3), category(1), float64(1), int64(4)
memory usage: 38.8 KB


In [164]:
# 타겟 변수와 피처 변수 설정
X = df.drop('Survived', axis=1)
y = df['Survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred)) # 예측 정확도
#print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
#print("Classification Report:\n", classification_report(y_test, y_pred))


(712, 8) (179, 8) (712,) (179,)
Accuracy: 0.8156424581005587
